In [3]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error


encoded_sales_df = pd.read_csv("encoded_NewData.csv")

# Define features and target variable
X = encoded_sales_df.drop(columns=["Sales"])  # Features
y = encoded_sales_df["Sales"]  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)
y_train_log = np.log1p(y_train)   # log(1 + y)
y_test_log = np.log1p(y_test)

# ----- 2. Train the XGBoost Regressor on log-transformed target -----
xgb_model = XGBRegressor(random_state=42, n_estimators=100, learning_rate=0.1, max_depth=6)
xgb_model.fit(X_train, y_train_log)

# ----- 3. Predict on the test data (in log scale) -----
y_pred_log = xgb_model.predict(X_test)

# ----- 4. Inverse the transformation to get real-scale predictions -----
y_pred = np.expm1(y_pred_log)  # inverse of log1p
y_test_real = np.expm1(y_test_log)

# ----- 5. Evaluate model performance -----
xgb_mse = mean_squared_error(y_test, y_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_mae = mean_absolute_error(y_test, y_pred)
xgb_rmsle = np.sqrt(mean_squared_log_error(y_test, np.maximum(0, y_pred)))  # clamp negatives if any

# ----- 6. Print results -----
results_xgb = pd.DataFrame({
    'Model': ['XGBoost (Log Target)'],
    'RMSLE': [xgb_rmsle],
    'RMSE': [xgb_rmse],
    'MSE': [xgb_mse],
    'MAE': [xgb_mae]
}).round(2)

print(results_xgb)

# ----- 7. R² Scores -----
print("Train R²:", xgb_model.score(X_train, y_train_log))  # still log-scale
print("Test R²:", xgb_model.score(X_test, y_test_log))

pickle.dump(xgb_model, open('model.pkl','wb'))
with open("columns.pkl", "wb") as f:
    pickle.dump(X_train.columns.tolist(), f)
# model = pickle.load(open('model.pkl','rb'))
# print(model.predict([[80, 1770000, 6000, 85]]))

Training set size: (1565, 30)
Testing set size: (392, 30)
                  Model  RMSLE   RMSE       MSE    MAE
0  XGBoost (Log Target)   0.48  132.2  17477.38  77.32
Train R²: 0.9713608215740697
Test R²: 0.8740284786794396
